<a href="https://colab.research.google.com/github/ChowchowWorks/Customer_service_rag/blob/main/Rag_Pipeline_Prototype_Version_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Import Libraries

In [1]:
import os
from google.colab import userdata

!pip install langchain_community langchain chromadb transformers sentence-transformers
!pip install -U langchain-huggingface
!pip install wikipedia
!pip install pypdf

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = "API_token"
os.environ['HUGGINGFACEHUB_API_TOKEN'] = "API_token"
os.environ['USER_AGENT'] = 'MyColabApp/1.0 (Python/3.9; GoogleColab)'

In [25]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import WikipediaLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.llms import HuggingFaceHub
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain import HuggingFacePipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.prompts import ChatPromptTemplate

embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


# Step 2: Loading Documents

(a) Load Documents

In [3]:
loader = WikipediaLoader(query="National University of Singapore", lang = 'en')
docs = loader.load()

(b) Splitting

In [15]:
splitter = RecursiveCharacterTextSplitter(chunk_size = 300, chunk_overlap = 50)
texts = splitter.split_documents(docs)

(d) LLM

In [16]:
llm = HuggingFaceHub(repo_id="mistralai/Mistral-7B-Instruct-v0.3", model_kwargs={"temperature":0.5, "max_length":1024})

# Step 3: Indexing

(a) Get Question

In [87]:
question = input("Ask me anything!\n")

Ask me anything!
List me the halls of NUS


(b) Count tokens

In [40]:
from transformers import AutoTokenizer
from huggingface_hub import login
login("API_token")

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")

def num_quest_tokens(question):
  return len(tokenizer.encode(question))

In [41]:
quest_tokens = num_quest_tokens(question)

(c) Embeddings

In [42]:
# Query Embeddings
query_result = embedding.embed_query(question)

# Document Embeddings
text_chunks = [doc.page_content for doc in texts]
document_result = embedding.embed_documents(text_chunks)

print(len(query_result))

384


(d) Cosine Similarity

In [43]:
import numpy as np

def cosine_similarity(vec1, vec2):
  v1dotv2 = np.dot(vec1, vec2)
  vec1_norm = np.linalg.norm(vec1)
  vec2_norm = np.linalg.norm(vec2)
  return v1dotv2 / (vec1_norm * vec2_norm)

print(cosine_similarity(query_result, document_result[0])) # not needed Chroma has this already

0.5455274981876062


(e) Consolidated into Chroma

In [89]:
vectorstore = Chroma.from_documents(texts, embedding)
retriever = vectorstore.as_retriever(search_kwargs = {'k' : 3})

# Step 4: Retrieval

In [90]:
relevent = retriever.invoke(question)
print(len(relevent))

3


# Step 5: Generation

(a) Prompt

In [46]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

(b) Hugging Face Inference Client

In [47]:
from huggingface_hub import InferenceClient
client = InferenceClient(model="mistralai/Mistral-7B-Instruct-v0.3", token= "API_token")

(c) Wrappers and Chat

In [96]:
from langchain_core.runnables import Runnable

class HuggingFaceChatRunnable(Runnable):
    def __init__(self, client, prompt_template, temperature, max_tokens):
        self.client = client
        self.prompt_template = prompt_template
        self.temperature = temperature
        self.max_tokens = max_tokens

    def invoke(self, inputs: dict) -> str:
        prompt_str = self.prompt_template.format(**inputs)

        response = self.client.chat_completion(
            messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt_str}],
            temperature = self.temperature,
            max_tokens = self.max_tokens)
        return response.choices[0].message["content"]

chat = HuggingFaceChatRunnable(client, prompt_template=prompt, temperature= 0.4, max_tokens= 1024)

(d) Response

In [98]:
response = chat.invoke({"context": relevent, "question": question})
print(response)

Based on the provided context, the documents do not contain information about the halls of the National University of Singapore (NUS). The documents mention the main campus located adjacent to the Kent Ridge subzone of Queenstown, the Duke–NUS Medical School located at the Outram campus, and the Bukit Timah campus, but they do not provide information about the specific halls or dormitories within these campuses.
